<a href="https://colab.research.google.com/github/AlinaTyub/Colab-projects/blob/master/SMS_3000_AB_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Описание:**  
21.11.23 Клиентам были отправлены смс по rfm сегментам с напоминанием про 3 000 бонусов. По каждому сегменту были отправлены по два вида смс в зависимости от того, анкетированный клиент или нет. То есть содержание смс немного отличались и ссылки тоже отличались. Задача — провести оценку результатов A/B-теста и сделать выводы об эффективности отправленных смс.

**Гипотеза:** Смс с напоминанием про 3000 бонусов приведет к увеличению кол-ва покупок за 7 дней (проверяется для каждого сегмента rfm, анкетированных и неакнкетированных клиентов отдельно).

H0: доля покупок за период 7 дней после СМС в тестовой и контрольной выборках равны.  
H1: доля покупок за период 7 дней после СМС в тестовой выборке значимо больше, чем в контрольной.

Для проверки гипотезы буду использовать критерий Хи-квадрат.

Группы: 0 — контрольная, 1 — отправили смс.  
Старт теста – 21.11.2023, конец – 27.11.2023.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as stats
import pyodbc
# from statsmodels.stats.power import zt_ind_solve_power
import statsmodels.stats.power as smp
import statsmodels.stats.proportion as proportion

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
anketir_133_test = pd.read_csv('133_anketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_133_test['group'] = 'anketir_133_1'

<ipython-input-7-82bbb0e889df>:1: DtypeWarning: Columns (42,43,44,45,63,81,82,83,84,90,91,94,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  anketir_133_test = pd.read_csv('133_anketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')


In [8]:
anketir_133_test.head()

,customer_mindbox_id,group
0,7812,anketir_133_1
1,13413,anketir_133_1
2,30847,anketir_133_1
3,32925,anketir_133_1
4,38816,anketir_133_1


In [9]:
anketir_133_nedostavleno = pd.read_csv('133_anketir_nedostavleno.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_133_nedostavleno['group'] = 'anketir_133_0'

In [10]:
anketir_133_nedostavleno.head()

,customer_mindbox_id,group
0,86263,anketir_133_0
1,120553,anketir_133_0
2,141682,anketir_133_0
3,170815,anketir_133_0
4,181794,anketir_133_0


In [11]:
anketir_133_test = anketir_133_test.merge(anketir_133_nedostavleno, on='customer_mindbox_id', how='left')
anketir_133_test = anketir_133_test[anketir_133_test['group_y'].isnull()]
anketir_133_test = anketir_133_test[['customer_mindbox_id', 'group_x']]

In [12]:
anketir_133_test.head()

,customer_mindbox_id,group_x
0,7812,anketir_133_1
1,13413,anketir_133_1
2,30847,anketir_133_1
3,32925,anketir_133_1
4,38816,anketir_133_1


In [13]:
anketir_133_test.dtypes

customer_mindbox_id     int64
group_x                object
dtype: object

In [15]:
len(anketir_133_test)

13331

In [16]:
anketir_133_control = pd.read_csv('133_anketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_133_control['group'] = 'anketir_133_0'

In [17]:
anketir_133_control = pd.concat([anketir_133_nedostavleno, anketir_133_control], ignore_index=True)

In [18]:
len(anketir_133_control)

3779

In [19]:
neanketir_133_test = pd.read_csv('133_neanketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_133_test['group'] = 'neanketir_133_1'

<ipython-input-19-16de97fae2ef>:1: DtypeWarning: Columns (29,42,43,44,45,54,55,56,57,63,77,78,79,80,90,91,92,93,94,95,96,102,105) have mixed types. Specify dtype option on import or set low_memory=False.
  neanketir_133_test = pd.read_csv('133_neanketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')


In [20]:
neanketir_133_nedostavleno = pd.read_csv('133_neanketir_nedostavleno.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_133_nedostavleno['group'] = 'neanketir_133_0'

In [21]:
neanketir_133_test = neanketir_133_test.merge(neanketir_133_nedostavleno, on='customer_mindbox_id', how='left')
neanketir_133_test = neanketir_133_test[neanketir_133_test['group_y'].isnull()]
neanketir_133_test = neanketir_133_test[['customer_mindbox_id', 'group_x']]

In [22]:
len(neanketir_133_test)

17345

In [23]:
neanketir_133_control = pd.read_csv('133_neanketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_133_control['group'] = 'neanketir_133_0'

In [24]:
neanketir_133_control = pd.concat([neanketir_133_nedostavleno, neanketir_133_control], ignore_index=True)

In [25]:
len(neanketir_133_control)

5800

In [26]:
anketir_233_test = pd.read_csv('233_anketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_233_test['group'] = 'anketir_233_1'

<ipython-input-26-380ae74515e3>:1: DtypeWarning: Columns (37,42,43,44,45,46,47,48,49,50,51,52,53,63,81,82,83,84,90,91,94,95,96,102) have mixed types. Specify dtype option on import or set low_memory=False.
  anketir_233_test = pd.read_csv('233_anketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')


In [27]:
anketir_233_nedostavleno = pd.read_csv('233_anketir_nedostavleno.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_233_nedostavleno['group'] = 'anketir_233_0'

In [28]:
anketir_233_test = anketir_233_test.merge(anketir_233_nedostavleno, on='customer_mindbox_id', how='left')
anketir_233_test = anketir_233_test[anketir_233_test['group_y'].isnull()]
anketir_233_test = anketir_233_test[['customer_mindbox_id', 'group_x']]

In [29]:
len(anketir_233_test)

31283

In [30]:
anketir_233_control = pd.read_csv('233_anketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_233_control ['group'] = 'anketir_233_0'

<ipython-input-30-71e83483ea8d>:1: DtypeWarning: Columns (42,43,44,45,63,90,91,94,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  anketir_233_control = pd.read_csv('233_anketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')


In [31]:
anketir_233_control = pd.concat([anketir_233_nedostavleno, anketir_233_control], ignore_index=True)

In [32]:
len(anketir_233_control)

6898

In [33]:
neanketir_233_test = pd.read_csv('233_neanketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_233_test['group'] = 'neanketir_233_1'

<ipython-input-33-90450c8b9e7d>:1: DtypeWarning: Columns (1,42,43,44,45,63,81,82,83,84,91,93,94,95,96,102,105) have mixed types. Specify dtype option on import or set low_memory=False.
  neanketir_233_test = pd.read_csv('233_neanketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')


In [34]:
neanketir_233_nedostavleno = pd.read_csv('233_neanketir_nedostavleno.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_233_nedostavleno['group'] = 'neanketir_233_0'

In [35]:
neanketir_233_test = neanketir_233_test.merge(neanketir_233_nedostavleno, on='customer_mindbox_id', how='left')
neanketir_233_test = neanketir_233_test[neanketir_233_test['group_y'].isnull()]
neanketir_233_test = neanketir_233_test[['customer_mindbox_id', 'group_x']]

In [36]:
len(neanketir_233_test)

13730

In [37]:
neanketir_233_control = pd.read_csv('233_neanketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_233_control['group'] = 'neanketir_233_0'

In [38]:
neanketir_233_control = pd.concat([neanketir_233_nedostavleno, neanketir_233_control], ignore_index=True)

In [39]:
len(neanketir_233_control)

3861

In [40]:
anketir_333_test = pd.read_csv('333_anketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_333_test['group'] = 'anketir_333_1'

<ipython-input-40-ca7099950944>:1: DtypeWarning: Columns (25,29,37,42,43,44,45,46,47,48,49,50,51,52,53,63,64,73,74,75,76,81,82,83,84,90,91,94,95,96,102) have mixed types. Specify dtype option on import or set low_memory=False.
  anketir_333_test = pd.read_csv('333_anketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')


In [41]:
anketir_333_nedostavleno = pd.read_csv('333_anketir_nedostavleno.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_333_nedostavleno['group'] = 'anketir_333_0'

In [42]:
anketir_333_test = anketir_333_test.merge(anketir_333_nedostavleno, on='customer_mindbox_id', how='left')
anketir_333_test = anketir_333_test[anketir_333_test['group_y'].isnull()]
anketir_333_test = anketir_333_test[['customer_mindbox_id', 'group_x']]


In [43]:
len(anketir_333_test)

35488

In [44]:
anketir_333_control = pd.read_csv('333_anketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
anketir_333_control['group'] = 'anketir_333_0'

<ipython-input-44-9617f31c2a44>:1: DtypeWarning: Columns (27,42,43,44,45,81,82,83,84,102) have mixed types. Specify dtype option on import or set low_memory=False.
  anketir_333_control = pd.read_csv('333_anketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')


In [45]:
anketir_333_control = pd.concat([anketir_333_nedostavleno, anketir_333_control], ignore_index=True)

In [46]:
len(anketir_333_control)

7007

In [47]:
neanketir_333_test = pd.read_csv('333_neanketir_test.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_333_test['group'] = 'neanketir_333_1'

In [48]:
neanketir_333_nedostavleno = pd.read_csv('333_neanketir_nedostavleno.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_333_nedostavleno['group'] = 'neanketir_333_0'

In [49]:
neanketir_333_test = neanketir_333_test.merge(neanketir_333_nedostavleno, on='customer_mindbox_id', how='left')
neanketir_333_test = neanketir_333_test[neanketir_333_test['group_y'].isnull()]
neanketir_333_test = neanketir_333_test[['customer_mindbox_id', 'group_x']]

In [50]:
len(neanketir_333_test)

2291

In [51]:
neanketir_333_control = pd.read_csv('333_neanketir_control.csv', sep=';')['CustomerIdsMindboxId'].to_frame (name='customer_mindbox_id')
neanketir_333_control['group'] = 'neanketir_333_0'

In [52]:
neanketir_333_control = pd.concat([neanketir_333_nedostavleno, neanketir_333_control], ignore_index=True)

In [53]:
len(neanketir_333_control)

534

In [54]:
def select(sql):
  return pd.read_sql(sql, con)

In [55]:
with open('sql_user.txt', 'r', encoding='utf-8-sig') as fp:
    sql_user = fp.read().rstrip()

with open('sql_password.txt', 'r', encoding='utf-8-sig') as fp:
    sql_password = fp.read().rstrip()

In [57]:
sql_password

'k@DKXyY2jeAY'

In [59]:
con = pyodbc.connect('DRIVER={SQL Server}; SERVER=DWH01; DATABASE=norbit_spl; UID=sql_user; PWD=sql_password')

Error: ignored

In [60]:
sql = '''
with  orders_and_max_datetime as(
    select o.customer_mindbox_id,
            o.order_mindbox_id,
            o.first_action_datetime,
            cast(o.first_action_datetime as date) as first_action_date,
            o.action_datetime,
            o.total_price,
            max(o.action_datetime) over (partition by o.order_mindbox_id) as max_action_datetime
    from dbo_mb.orders o
    where cast(first_action_datetime as date) >= '2023-11-21'
    and cast(first_action_datetime as date) <= '2023-11-27'
    and brand = 'reStore'
    )
select distinct o.customer_mindbox_id,
        o.order_mindbox_id,
        o.first_action_datetime,
        first_action_date,
        o.total_price
from orders_and_max_datetime o
where o.action_datetime = o.max_action_datetime
and o.total_price > 0
'''

In [ ]:
orders = select (sql)

In [ ]:
orders.head()

#Анкетированные 133
Размер тестовой выборки (смс отправлены и доставлены): 13 331  
Потратили на смс: 46 341,438  
Размер контрольной выборки: 3 779

In [65]:
nobs = 3779 #кол-во наблюдений
alpha = 0.01

In [ ]:
anketir_133_test = anketir_133_test.merge(orders, on='customer_mindbox_id', how='left')
anketir_133_test.head()

In [63]:
def buy_or_not(orders_cnt):
  if orders_cnt == 0:
    buy = 0
  else:
    buy = 1
  return buy

In [ ]:
# Определяем наличие или отсутствие покупки для каждого клиента
anketir_133_test_group = anketir_133_test.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
anketir_133_test_group = anketir_133_test_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
anketir_133_test_group['buy'] = anketir_133_test_group['orders_cnt'].apply(buy_or_not)
anketir_133_test_group['group'] = 'anketir_133_1'
anketir_133_test_group.head()

In [ ]:
anketir_133_test['order_mindbox_id'].count()

In [ ]:
# Кол-во покупок на клиента
round(anketir_133_test['order_mindbox_id'].count() / anketir_133_test['customer_mindbox_id'].nunique () , 3)

In [ ]:
# Выручка с одного клиента (считаются все клиенты, которые получили смс, не обязательно сделавшие заказ):
ARPU1 = round(anketir_133_test['total_price'].sum() / anketir_133_test['customer_mindbox_id'].nunique () , 1)
ARPU1

In [ ]:
# Доля купивших
p1 = round(anketir_133_test_group['buy'].sum() / anketir_133_test_group['customer_mindbox_id'].nunique () , 3)
p1

In [ ]:
anketir_133_control = anketir_133_control.merge(orders, on='customer_mindbox_id', how='left')
anketir_133_control.head()

In [ ]:
anketir_133_control_group = anketir_133_control.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
anketir_133_control_group = anketir_133_control_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
anketir_133_control_group['buy'] = anketir_133_control_group['orders_cnt'].apply(buy_or_not)
anketir_133_control_group['group'] = 'anketir_133_0'
anketir_133_control_group.head()

In [ ]:
# Кол-во покупок на клиента
round(anketir_133_control['total_price'].count() / anketir_133_control['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU0 = round(anketir_133_control['total_price'].sum() / anketir_133_control['customer_mindbox_id'].nunique () , 1)
ARPU0

In [ ]:
p0 = round(anketir_133_control_group['buy'].sum() / anketir_133_control_group['customer_mindbox_id'].nunique () , 3)
p0

In [ ]:
effect_size = ((p0- p1)**2 / p0) ** 0.5
effect_size

In [64]:
chipower = smp.GofChisquarePower()

In [ ]:
# Вычислим мощность теста
power = chipower.solve_power(effect_size = ((p0- p1)**2 / p0) ** 0.5,
                    nobs = nobs,
                    alpha = alpha,
                    power = None)
power

In [ ]:
anketir_133 = pd.concat([anketir_133_test_group, anketir_133_control_group], ignore_index=True)
anketir_133 = anketir_133.groupby('group').agg({'customer_mindbox_id': 'count', 'buy': 'sum'}).reset_index() \
                .rename(columns ={'customer_mindbox_id': 'clients_count', 'buy': 'buyers_count'})
anketir_133.head()

In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(anketir_133['buyers_count'], anketir_133['clients_count'])

In [ ]:
pval

In [ ]:
pval < alpha

Доля покупок за период 7 дней после СМС в тестовой выборке среди анкетированных 133 сегмента значимо больше, чем в контрольной, при достоверности 99% и мощности 79%.

# Неанкетированные 133
Размер тестовой выборки (смс отправлены и доставлены): 17 345  
Потратили на смс: 62 730,312  
Размер контрольной выборки: 5 800

In [66]:
nobs = 5800 #кол-во наблюдений
alpha = 0.01

In [ ]:
neanketir_133_test = neanketir_133_test.merge(orders, on='customer_mindbox_id', how='left')
neanketir_133_test.head()

In [ ]:
neanketir_133_test_group = neanketir_133_test.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
neanketir_133_test_group = neanketir_133_test_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
neanketir_133_test_group['buy'] = neanketir_133_test_group['orders_cnt'].apply(buy_or_not)
neanketir_133_test_group['group'] = 'neanketir_133_1'

In [ ]:
# Кол-во покупок на клиента
round(neanketir_133_test['order_mindbox_id'].count() / neanketir_133_test['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU1 = round(neanketir_133_test['total_price'].sum() / neanketir_133_test['customer_mindbox_id'].nunique () , 1)
ARPU1

In [ ]:
# Доля купивших
p1 = round(neanketir_133_test_group['buy'].sum() / neanketir_133_test_group['customer_mindbox_id'].nunique () , 3)
p1

In [ ]:
neanketir_133_control = neanketir_133_control.merge(orders, on='customer_mindbox_id', how='left')
neanketir_133_control.head()

In [ ]:
neanketir_133_control_group = neanketir_133_control.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
neanketir_133_control_group = neanketir_133_control_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
neanketir_133_control_group['buy'] = neanketir_133_control_group['orders_cnt'].apply(buy_or_not)
neanketir_133_control_group['group'] = 'neanketir_133_0'

In [ ]:
# Кол-во покупок на клиента
round(neanketir_133_control['total_price'].count() / neanketir_133_control['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU0 = round(neanketir_133_control['total_price'].sum() / neanketir_133_control['customer_mindbox_id'].nunique () , 1)
ARPU0

In [ ]:
p0 = round(neanketir_133_control_group['buy'].sum() / neanketir_133_control_group['customer_mindbox_id'].nunique () , 3)
p0

In [ ]:
effect_size = ((p0- p1)**2 / p0) ** 0.5
effect_size

In [ ]:
# Вычислим мощность теста
power = chipower.solve_power(effect_size = ((p0- p1)**2 / p0) ** 0.5,
                    nobs = nobs,
                    alpha = alpha,
                    power = None)
power

In [ ]:
neanketir_133 = pd.concat([neanketir_133_test_group, neanketir_133_control_group], ignore_index=True)
neanketir_133 = neanketir_133.groupby('group').agg({'customer_mindbox_id': 'count', 'buy': 'sum'}).reset_index() \
                .rename(columns ={'customer_mindbox_id': 'clients_count', 'buy': 'buyers_count'})
neanketir_133.head()

In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(neanketir_133['buyers_count'], neanketir_133['clients_count'])

In [ ]:
pval

In [ ]:
pval < alpha

Статистически значимой разницы в доле продаж среди неанкетированных 133 сегмента не обнаружено, так как p-value выше выбранного порога ошибки первого рода при мощности теста 80%.

# Анкетированные 233
Размер тестовой выборки (смс отправлены и доставлены): 31 283  
Потратили на смс: 103 031,329  
Размер контрольной выборки: 6 898

In [67]:
nobs = 6898 #кол-во наблюдений
alpha = 0.01

In [ ]:
anketir_233_test = anketir_233_test.merge(orders, on='customer_mindbox_id', how='left')
anketir_233_test.head()

In [ ]:
anketir_233_test_group = anketir_233_test.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
anketir_233_test_group = anketir_233_test_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
anketir_233_test_group['buy'] = anketir_233_test_group['orders_cnt'].apply(buy_or_not)
anketir_233_test_group['group'] = 'anketir_233_1'

In [ ]:
# Кол-во покупок на клиента
round(anketir_233_test['order_mindbox_id'].count() / anketir_233_test['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU1 = round(anketir_233_test['total_price'].sum() / anketir_233_test['customer_mindbox_id'].nunique () , 1)
ARPU1

In [ ]:
# Доля купивших
p1 = round(anketir_233_test_group['buy'].sum() / anketir_233_test_group['customer_mindbox_id'].nunique () , 3)
p1

In [ ]:
anketir_233_control = anketir_233_control.merge(orders, on='customer_mindbox_id', how='left')
anketir_233_control.head()

In [ ]:
anketir_233_control_group = anketir_233_control.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
anketir_233_control_group = anketir_233_control_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
anketir_233_control_group['buy'] = anketir_233_control_group['orders_cnt'].apply(buy_or_not)
anketir_233_control_group['group'] = 'anketir_233_0'

In [ ]:
# Кол-во покупок на клиента
round(anketir_233_control['total_price'].count() / anketir_233_control['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU0 = round(anketir_233_control['total_price'].sum() / anketir_233_control['customer_mindbox_id'].nunique () , 1)
ARPU0

In [ ]:
p0 = round(anketir_233_control_group['buy'].sum() / anketir_233_control_group['customer_mindbox_id'].nunique () , 3)
p0

In [ ]:
effect_size = ((p0- p1)**2 / p0) ** 0.5
effect_size

In [ ]:
# Вычислим мощность теста
power = chipower.solve_power(effect_size = ((p0- p1)**2 / p0) ** 0.5,
                    nobs = nobs,
                    alpha = alpha,
                    power = None)
power

In [ ]:
anketir_233 = pd.concat([anketir_233_test_group, anketir_233_control_group], ignore_index=True)
anketir_233 = anketir_233.groupby('group').agg({'customer_mindbox_id': 'count', 'buy': 'sum'}).reset_index() \
                .rename(columns ={'customer_mindbox_id': 'clients_count', 'buy': 'buyers_count'})
anketir_233.head()

In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(anketir_233['buyers_count'], anketir_233['clients_count'])

In [ ]:
pval

In [ ]:
pval < alpha

Доля покупок за период 7 дней после СМС в тестовой выборке среди анкетиорованных 233 сегмента значимо больше, чем в контрольной, при достоверности 99%.

# Неанкетированные 233
Размер тестовой выборки (смс отправлены и доставлены): 13 730  
Потратили на смс: 47 424,538  
Размер контрольной выборки: 3 861

In [68]:
nobs = 3861 #кол-во наблюдений
alpha = 0.01

In [ ]:
neanketir_233_test = neanketir_233_test.merge(orders, on='customer_mindbox_id', how='left')
neanketir_233_test.head()

In [ ]:
neanketir_233_test_group = neanketir_233_test.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
neanketir_233_test_group = neanketir_233_test_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
neanketir_233_test_group['buy'] = neanketir_233_test_group['orders_cnt'].apply(buy_or_not)
neanketir_233_test_group['group'] = 'neanketir_233_1'

In [ ]:
# Кол-во покупок на клиента
round(neanketir_233_test['order_mindbox_id'].count() / neanketir_233_test['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU1 = round(neanketir_233_test['total_price'].sum() / neanketir_233_test['customer_mindbox_id'].nunique () , 1)
ARPU1

In [ ]:
# Доля купивших
p1 = round(neanketir_233_test_group['buy'].sum() / neanketir_233_test_group['customer_mindbox_id'].nunique () , 3)
p1

In [ ]:
neanketir_233_control = neanketir_233_control.merge(orders, on='customer_mindbox_id', how='left')
neanketir_233_control.head()

In [ ]:
neanketir_233_control_group = neanketir_233_control.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
neanketir_233_control_group = neanketir_233_control_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
neanketir_233_control_group['buy'] = neanketir_233_control_group['orders_cnt'].apply(buy_or_not)
neanketir_233_control_group['group'] = 'neanketir_233_0'

In [ ]:
# Кол-во покупок на клиента
round(neanketir_233_control['total_price'].count() / neanketir_233_control['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU0 = round(neanketir_233_control['total_price'].sum() / neanketir_233_control['customer_mindbox_id'].nunique () , 1)
ARPU0

In [ ]:
p0 = round(neanketir_233_control_group['buy'].sum() / neanketir_233_control_group['customer_mindbox_id'].nunique () , 3)
p0

In [ ]:
effect_size = ((p0- p1)**2 / p0) ** 0.5
effect_size

In [ ]:
# Вычислим мощность теста
power = chipower.solve_power(effect_size = ((p0- p1)**2 / p0) ** 0.5,
                    nobs = nobs,
                    alpha = alpha,
                    power = None)
power

In [ ]:
neanketir_233 = pd.concat([neanketir_233_test_group, neanketir_233_control_group], ignore_index=True)
neanketir_233 = neanketir_233.groupby('group').agg({'customer_mindbox_id': 'count', 'buy': 'sum'}).reset_index() \
                .rename(columns ={'customer_mindbox_id': 'clients_count', 'buy': 'buyers_count'})
neanketir_233.head()

In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(neanketir_233['buyers_count'], neanketir_233['clients_count'])

In [ ]:
pval

In [ ]:
pval < alpha

Доля покупок за период 7 дней после СМС в тестовой выборке среди неанкетированных 233 сегмента значимо больше, чем в контрольной, при достоверности 99%.

# Анкетированные 333
Размер тестовой выборки (смс отправлены и доставлены): 35 488  
Потратили на смс 114 280,483  
Размер контрольной выборки 7 007

In [ ]:
nobs = 7007 #кол-во наблюдений
alpha = 0.01

In [ ]:
anketir_333_test = anketir_333_test.merge(orders, on='customer_mindbox_id', how='left')
anketir_333_test.head()

In [ ]:
anketir_333_test_group = anketir_333_test.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
anketir_333_test_group = anketir_333_test_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
anketir_333_test_group['buy'] = anketir_333_test_group['orders_cnt'].apply(buy_or_not)
anketir_333_test_group['group'] = 'anketir_333_1'

In [ ]:
# Кол-во покупок на клиента
round(anketir_333_test['order_mindbox_id'].count() / anketir_333_test['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU1 = round(anketir_333_test['total_price'].sum() / anketir_333_test['customer_mindbox_id'].nunique () , 1)
ARPU1

In [ ]:
# Доля купивших
p1 = round(anketir_333_test_group['buy'].sum() / anketir_333_test_group['customer_mindbox_id'].nunique () , 3)
p1

In [ ]:
anketir_333_control = anketir_333_control.merge(orders, on='customer_mindbox_id', how='left')
anketir_333_control.head()

In [ ]:
anketir_333_control_group = anketir_333_control.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
anketir_333_control_group = anketir_333_control_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
anketir_333_control_group['buy'] = anketir_333_control_group['orders_cnt'].apply(buy_or_not)
anketir_333_control_group['group'] = 'anketir_333_0'

In [ ]:
# Кол-во покупок на клиента
round(anketir_333_control['total_price'].count() / anketir_333_control['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU0 = round(anketir_333_control['total_price'].sum() / anketir_333_control['customer_mindbox_id'].nunique () , 1)
ARPU0

In [ ]:
p0 = round(anketir_333_control_group['buy'].sum() / anketir_333_control_group['customer_mindbox_id'].nunique () , 3)
p0

In [ ]:
effect_size = ((p0- p1)**2 / p0) ** 0.5
effect_size

In [ ]:
# Вычислим мощность теста
power = chipower.solve_power(effect_size = ((p0- p1)**2 / p0) ** 0.5,
                    nobs = nobs,
                    alpha = alpha,
                    power = None)
power

In [ ]:
anketir_333 = pd.concat([anketir_333_test_group, anketir_333_control_group], ignore_index=True)
anketir_333 = anketir_333.groupby('group').agg({'customer_mindbox_id': 'count', 'buy': 'sum'}).reset_index() \
                .rename(columns ={'customer_mindbox_id': 'clients_count', 'buy': 'buyers_count'})
anketir_333.head()

In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(anketir_333['buyers_count'], anketir_333['clients_count'])

In [ ]:
pval

In [ ]:
pval < alpha

Доля покупок за период 7 дней после СМС в тестовой выборке среди анкетированных 333 сегмента значимо больше, чем в контрольной, при достоверности 99%.

# Неанкетированные 333
Отправлено смс 2 291  
Потратили на смс 7 683,983  
Размер контрольной выборки 534

In [69]:
nobs = 534 #кол-во наблюдений
alpha = 0.01

In [ ]:
neanketir_333_test = neanketir_333_test.merge(orders, on='customer_mindbox_id', how='left')
neanketir_333_test.head()

In [ ]:
neanketir_333_test_group = neanketir_333_test.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
neanketir_333_test_group = neanketir_333_test_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
neanketir_333_test_group['buy'] = neanketir_333_test_group['orders_cnt'].apply(buy_or_not)
neanketir_333_test_group['group'] = 'neanketir_333_1'

In [ ]:
# Кол-во покупок на клиента
round(neanketir_333_test['order_mindbox_id'].count() / neanketir_333_test['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU1 = round(neanketir_333_test['total_price'].sum() / neanketir_333_test['customer_mindbox_id'].nunique () , 1)
ARPU1

In [ ]:
# Доля купивших
p1 = round(neanketir_333_test_group['buy'].sum() / neanketir_333_test_group['customer_mindbox_id'].nunique () , 3)
p1

In [ ]:
neanketir_333_control = neanketir_333_control.merge(orders, on='customer_mindbox_id', how='left')
neanketir_333_control.head()

In [ ]:
neanketir_333_control_group = neanketir_333_control.groupby('customer_mindbox_id').agg({'order_mindbox_id': 'count'})
neanketir_333_control_group = neanketir_333_control_group.reset_index().rename(columns ={'order_mindbox_id': 'orders_cnt'})
neanketir_333_control_group['buy'] = neanketir_333_control_group['orders_cnt'].apply(buy_or_not)
neanketir_333_control_group['group'] = 'neanketir_333_0'

In [ ]:
# Кол-во покупок на клиента
round(neanketir_333_control['total_price'].count() / neanketir_333_control['customer_mindbox_id'].nunique () , 3)

In [ ]:
ARPU0 = round(neanketir_333_control['total_price'].sum() / neanketir_333_control['customer_mindbox_id'].nunique () , 1)
ARPU0

In [ ]:
p0 = round(neanketir_333_control_group['buy'].sum() / neanketir_333_control_group['customer_mindbox_id'].nunique () , 3)
p0

In [ ]:
effect_size = ((p0- p1)**2 / p0) ** 0.5
effect_size

In [ ]:
# Вычислим мощность теста
power = chipower.solve_power(effect_size = ((p0- p1)**2 / p0) ** 0.5,
                    nobs = nobs,
                    alpha = alpha,
                    power = None)
power

In [ ]:
neanketir_333 = pd.concat([neanketir_333_test_group, neanketir_333_control_group], ignore_index=True)
neanketir_333 = neanketir_333.groupby('group').agg({'customer_mindbox_id': 'count', 'buy': 'sum'}).reset_index() \
                .rename(columns ={'customer_mindbox_id': 'clients_count', 'buy': 'buyers_count'})
neanketir_333.head()

In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(neanketir_333['buyers_count'], neanketir_333['clients_count'])

In [ ]:
pval

In [ ]:
pval < alpha

Статистически значимой разницы в доле продаж среди неанкетированных 333 сегмента не обнаружено, так как p-value выше выбранного порога ошибки первого рода при мощности теста 80%.

**Вывод:** смс с напоминанием про 3000 бонусов была эффективна по доле продаж для всех сегментов кроме неанкетированных 133 и неанкетированных 333.